In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import time
from sklearn.preprocessing import RobustScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.model_selection import KFold

from lib import data_generation as dg
from lib import feature_processing as fp

## 1. Data
### 1.0 Loading Data
Here we load the raw data

In [2]:
data = pd.read_csv('input/AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


### 1.1 Processing Data
The data is processed as explained in the data processing notebook of this project. The process function in feature_processing library is written based on that notebook.

In [3]:
processed_data = fp.process(data)

/Users/jacobsakhnini/Development/Anaconda/anaconda3/envs/airbnbEnv/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [4]:
processed_data.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,all_year_avail,low_avail,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodrow,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,40.64749,-73.97237,5.010635,1,9,2762,0.21,6,True,False,...,0,0,0,0,0,0,0,0,1,0
1,40.75362,-73.98377,5.420535,1,45,2976,0.38,2,True,False,...,0,0,0,0,0,0,0,1,0,0
2,40.80902,-73.94190,5.017280,3,0,0,0.00,1,True,False,...,0,0,0,0,0,0,0,0,1,0
3,40.68514,-73.95976,4.499810,1,270,3021,4.64,1,False,False,...,0,0,0,0,0,0,0,1,0,0
4,40.79851,-73.94399,4.394449,10,9,2793,0.10,1,False,True,...,0,0,0,0,0,0,0,1,0,0


### 1.2 Split Data
We split the data 80-20 into training and testing data

In [5]:
y = processed_data.price
processed_data = processed_data.drop(['price'], axis=1)

X = np.asarray(processed_data)
y = np.asarray(y).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training Dataset: {}".format(X_train.shape))
print("Testing Dataset: {}".format(X_test.shape))

Training Dataset: (39014, 239)
Testing Dataset: (9754, 239)


In [6]:
# data_copy = processed_data.copy()
# train_set = data_copy.sample(frac=0.8, random_state=0)
# test_set = data_copy.drop(train_set.index)

We convert them to numpy arrays for computation

In [7]:
# train_data = np.asarray(train_set).astype(np.float32)
# test_data = np.asarray(test_set).astype(np.float32)

### 1.2 Scaling Data
The final step in data processing is using the robust scaler to scale all data!

In [8]:
scaler = RobustScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

## 2. ML Models
In this project we tested multiple regressors at varying parameters. We concluded that the ideal model parameters were as follows for the top 3 regressors:

1) Random Forest: Number of estimators = 50  
2) Ridge Regression: Alpha = 5  
3) Huber Regression: Alpha = 10, Epsilon = 3  

As such, we will define our final models:

In [9]:
randomForest_final = RandomForestRegressor(n_estimators=50)
ridge_final = Ridge(alpha=5)
huber_final = HuberRegressor(alpha=10, epsilon=3)

In [10]:
# # Random Forest
# randomForest_final.fit(X_train,y_train)
# rf_y = randomForest_final.predict(X_test)
# rf_result = mean_squared_error(y_test, rf_y)
# print("Random Forest: {}".format(rf_result))

# # Ridge
# ridge_final.fit(X_train,y_train)
# ridge_y = ridge_final.predict(X_test)
# ridge_result = mean_squared_error(y_test, ridge_y)

# print("Ridge : {}".format(ridge_result))

# # Huber
# huber_final.fit(X_train,y_train)
# huber_y = huber_final.predict(X_test)
# huber_result = mean_squared_error(y_test,huber_y)
# print("Huber: {}".format(huber_result))

## 2.1 Unit Testing
We perform unit testing to check which samples perform the worst.

In [11]:
def unitTest(model, sampleX, sampleY):
    y_pred = model.predict(sampleX)
    return abs(sampleY - y_pred)

In [12]:
def getMinority(model, X, y, numFolds = 5, percent = 0.1):
    # Split training data into chunks
    cv = KFold(numFolds, True, 1)

    round = 1
    
    numSamples = int( (percent * X.shape[0]) / numFolds)
    
    # We test the model on each chunk after training on the other chunks
    for train, test in cv.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

        model.fit(X_train, y_train)
        
        diff_list = []
        for i in range(X_test.shape[0]):
            sampleX = X_test[i]
            sampleY = y_test[i]
            
            temp = unitTest(model, sampleX.reshape(1,-1), sampleY.reshape(-1))
            diff_list.append(temp)
            
        # Get poorest performance test values (highest mse)
        highest_diff = sorted(range(len(diff_list)), key=lambda i: diff_list[i])[-numSamples:]
        
        if (round==1):
            X_minority = X_test[highest_diff]
            y_minority = y_test[highest_diff]
        else:
            X_minority = np.concatenate( (X_minority, X_test[highest_diff]), axis = 0)
            y_minority = np.concatenate( (y_minority, y_test[highest_diff]), axis = 0)
            
        round = round + 1
 
    
    return X_minority, y_minority

In [13]:
X_m, y_m = getMinority(randomForest_final, X_train, y_train, percent=0.01)
print("Minority data acquired! X-shape = {}  ,  y-shape = {} ".format(X_m.shape, y_m.shape))

/Users/jacobsakhnini/Development/Anaconda/anaconda3/envs/airbnbEnv/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Minority data acquired! X-shape = (390, 239)  ,  y-shape = (390,) 


## 3. Generating Additional Data
We use the Synthetic Minority Oversampling Technique (SMOTE) to generate additional data from the training set.

In [14]:
minority_data = np.concatenate( (X_m, y_m.reshape(-1,1)), axis = 1)
print("Minority Data Shape: {}".format(minority_data.shape))

Minority Data Shape: (390, 240)


In [15]:
def genData(minority_data, N):
    newData = dg.SMOTE(minority_data, 100, 2)

    mse = mean_squared_error(minority_data, newData)
    r2 = r2_score(minority_data, newData)
    mae = mean_absolute_error(minority_data, newData)

#     print("Round 1/{}".format(N))
#     print("New data generated with MSE = {} , R2 = {} , MAE = {}".format(mse,r2,mae))
#     print("Size of training data = {}".format(minority_data.shape))
#     print("Size of new data = {}".format(newData.shape))

    for n in range(N-1):
        temp = dg.SMOTE(minority_data, 100, 2)

        mse = mean_squared_error(minority_data, temp)
        r2 = r2_score(minority_data, temp)
        mae = mean_absolute_error(minority_data, temp)

        newData = np.concatenate((newData, temp), axis = 0)

#         print("Round {}/{}".format(n+2,N))
#         print("New data generated with MSE = {} , R2 = {} , MAE = {}".format(mse,r2,mae))
#         print("Size of training data = {}".format(minority_data.shape))
#         print("Size of new data = {}".format(newData.shape))
        
    return newData

In [16]:
newData = genData(minority_data, 3)

As we can see from above, we created a new dataset that is similar to the minority set!

In [17]:
def combineData(X_train, y_train, newData):
#     print("Train data before new data: ")
#     print("Training Dataset: {}".format(X_train.shape))
#     print("Training Labels: {}".format(y_train.shape))

    y_m = newData[:,-1]
    X_m = np.delete(newData, np.s_[-1:], axis=1)
#     print("X_m shape = {}".format(X_m.shape))
#     print("y_m shape = {}".format(y_m.shape))

    X_train = np.concatenate((X_train, X_m), axis = 0)
    y_train = np.concatenate((y_train, y_m), axis = 0)

#     print("Train data after new data is added: ")
#     print("Training Dataset: {}".format(X_train.shape))
#     print("Training Labels: {}".format(y_train.shape))
    
    return X_train, y_train

In [18]:
X_train_new, y_train_new = combineData(X_train, y_train, newData)

## 4. ML Models With New Data

In [19]:
def testModel( model, X_train, y_train, X_test, y_test):
    # Random Forest
    model.fit(X_train,y_train)
    rf_y = model.predict(X_test)
    result = mean_squared_error(y_test, rf_y)
    print("Result: {}".format(result))
    
    return result

In [20]:
rf_result = testModel( randomForest_final , X_train_new, y_train_new, X_test, y_test)

Result: 0.17634252451509624


The results are better with more minority data in the training set!

### 4.1 Testing with more data
Previously, we generated 3 similar sets to the 10% minority (10% of the worst performance samples in the training set). Now we are going to test the same thing but with different percet minority and different number of similar sets (as opposed to 10% and 3 respectively).

In [21]:
percentages = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5]
N_values = [1,2,3,5,10]
model = randomForest_final

mse_final = []
p_final = []
n_final = []

for p in percentages:
    for n in N_values:
        print("P = {} , N = {}".format(p,n))
        X_m, y_m = getMinority(model, X_train, y_train, percent = p)
        minority_data = np.concatenate( (X_m, y_m.reshape(-1,1)), axis = 1)
        newData = genData(minority_data, n)
        X_train_new, y_train_new = combineData(X_train, y_train, newData)
        
        rf_result = testModel( randomForest_final , X_train_new, y_train_new, X_test, y_test)
        p_final.append(p)
        n_final.append(n)
        mse_final.append(rf_result)

P = 0.01 , N = 1


/Users/jacobsakhnini/Development/Anaconda/anaconda3/envs/airbnbEnv/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Result: 0.17575330862654054
P = 0.01 , N = 2


/Users/jacobsakhnini/Development/Anaconda/anaconda3/envs/airbnbEnv/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


KeyboardInterrupt: 

In [ ]:
finalResult_df = pd.DataFrame({
    'Percentage': p_final,
    'Num of extra sets': n_final,
    'MSE': mse_final,
})

finalResult_df.to_csv('output/results_dataGen_pn.csv', index = False, header=True)

In [ ]:
finalResult_df = finalResult_df.sort_values('MSE')

In [ ]:
finalResult_df.head(10)